Once you trained your model, you can use this code to generate the output.
For the tokenisation and embedding I used pre-trained model like BERT, which will help you to save timeand improve the accuracy of your model. You can use any other pre-trained model like RoBERTa.

In [ ]:
import torch
import pandas as pd
from transformers import BertTokenizer  

model_path = "Text_summarisation_model_part1.pt"
checkpoint = torch.load(model_path, map_location=torch.device('cuda' if torch.cuda.is_available() else 'cpu'))
model_config = checkpoint["model_config"]

bert_model_name = model_config["bert_model_name"]
tokenizer = BertTokenizer.from_pretrained(bert_model_name)

model_config_for_init = {k: v for k, v in model_config.items() if k != 'bert_model_name'}
model = TransformerSummarizer(**model_config_for_init)  
model.load_state_dict(checkpoint["model_state_dict"])
model.eval() 

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

In [ ]:
def prepare_input(text, tokenizer, max_seq_length=256):
    """Encodes the input text using the tokenizer."""
    src_encoding = tokenizer(text, max_length=max_seq_length,
                              truncation=True, padding="max_length", return_tensors="pt")
    src_ids = src_encoding["input_ids"].squeeze(0).to(device)
    return src_ids

In [ ]:
def generate_summary(model, text, tokenizer, max_length=50):
    """Generates a summary for the given text."""
    input_ids = prepare_input(text, tokenizer)
    generated_ids = model.generate(input_ids, max_length=max_length, beam_width=5,
                                   start_token_id=tokenizer.cls_token_id,
                                   end_token_id=tokenizer.sep_token_id,
                                   device=device)
    summary = tokenizer.decode(generated_ids, skip_special_tokens=True)
    return summary

In [ ]:
input_text = "" #Your input
summary = generate_summary(model, input_text, tokenizer)
print("Generated Summary:", summary)